# Cleaning the Music Tag Dataset

# 1. Introduction

__Background__ <br>
The data used in this notebook was scraped from www.bensound.com, a website for free-to-use music under the creative commons license. The dataset contains the url and tags for 253 tracks. <br>
__Aim__ <br>
In this notebook, we'll explore ways to clean the dataset. We'll extract the track names from the URL and use them for the index rows, convert the lists of tags from string to list objects, and correct misspelled tags.

# 2. Cleaning

## 2.1 Load Data

In [66]:
import pandas as pd

In [67]:
# set wd and filename
directory = "C:/Users/maxhi/OneDrive/Uni & Work/Programming/Data Science/Music Tagging/Data"
filename = "music_tags_raw.csv"
# load data
music_tags = pd.read_csv("{directory}/{filename}".format(
    directory = directory, filename = filename), sep = ";")

In [68]:
music_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 2 columns):
URL     253 non-null object
Tags    253 non-null object
dtypes: object(2)
memory usage: 2.0+ KB


In [69]:
music_tags.head()

,URL,Tags
0,https://www.bensound.com/royalty-free-music/tr...,"['ukulele', 'happy', 'funny', 'advertising', '..."
1,https://www.bensound.com/royalty-free-music/tr...,"['corporate', 'motivation', 'background', 'pre..."
2,https://www.bensound.com/royalty-free-music/tr...,"['rock', 'uplifting', 'success', 'positive', '..."
3,https://www.bensound.com/royalty-free-music/tr...,"['kid', 'kids', 'corporate', 'bouncy', 'happy'..."
4,https://www.bensound.com/royalty-free-music/tr...,"['jazz', 'jazzy', 'acoustic', 'old', 'light', ..."


## 2.2 Create Uniform Column Names

In [70]:
# set both columns to lowercase
new_columns = []
for column in music_tags.columns:
    new_columns.append(column.lower())
music_tags.columns = new_columns

In [71]:
music_tags.columns

Index(['url', 'tags'], dtype='object')

## 2.3 Extract Track Names from URL

There are two reasons why it is useful to extract the track names from the url. <br>
1) The track name is a piece of information similar to tags. There might be relationsships between track name and tags that we could explore. <br>
2) Pandas offers labeled indexing of columns and rows. We could set the track names as the row indices to select individual tracks in an intuitive way. <br>
Luckily, the url contains the track name at the same place. We can therefore extract them with string slicing.

In [72]:
# create new column "track_name"
# the track names start at index 50 in every url
music_tags["track_name"] = music_tags["url"].str.slice(start = 50)

In [73]:
# replace "-" with space
music_tags["track_name"] = music_tags["track_name"].str.replace("-", " ")

In [74]:
music_tags["track_name"].head()

0            ukulele
1     creative minds
2    a new beginning
3        little idea
4      jazzy frenchy
Name: track_name, dtype: object

## 2.4 Change Taglists from String to List

In [75]:
music_tags["tags"].dtype

dtype('O')

In [76]:
type(music_tags["tags"][0])

str

The taglists were scraped as list objects, but were read from the .csv file as strings. Let's change that.

In [77]:
# create empty list
tag_lists = []

# loop through
for string in music_tags["tags"]:
    string = eval(string)
    tag_lists.append(string)
    
music_tags["tags"] = tag_lists

In [78]:
music_tags.head()

,url,tags,track_name
0,https://www.bensound.com/royalty-free-music/tr...,"[ukulele, happy, funny, advertising, upbeat, k...",ukulele
1,https://www.bensound.com/royalty-free-music/tr...,"[corporate, motivation, background, presentati...",creative minds
2,https://www.bensound.com/royalty-free-music/tr...,"[rock, uplifting, success, positive, hope, hop...",a new beginning
3,https://www.bensound.com/royalty-free-music/tr...,"[kid, kids, corporate, bouncy, happy, upbeat, ...",little idea
4,https://www.bensound.com/royalty-free-music/tr...,"[jazz, jazzy, acoustic, old, light, retro, swi...",jazzy frenchy


## 2.5 Remove "\xa0" from Tag Strings

When we try to get all unique tags and their frequencies...

In [79]:
all_tags = {}

# Loop through every list
for tag_list in music_tags["tags"]:
    
    # Loop through every element in the list
    for tag in tag_list:
        # Add them to the dict or increment their frequency by 1
        if tag not in all_tags:
            all_tags[tag] = 1
        else:
            all_tags[tag] += 1

...we see some tags with "\xa0" in front of the actual tag. This is a unicode representation of a space. We need to remove these unicodes, so that typos aren't registered as different tags.

In [80]:

# Loop through all lists and their indices
for i, row in enumerate(music_tags["tags"]):
    
    # Setup an empty list for the cleaned tags
    tag_list = []
    
    # Look at ever individual tag
    for tag in row:
        
        # If \xa0 is in the tag, slice the string - remember, \xa0 means space
        if "\xa0" in tag:
            new_tag = tag[1:]
            tag_list.append(new_tag)
        else:
            tag_list.append(tag)
        
        # Reassign the new_tags to the df
    music_tags.iloc[i, 1] = tag_list

Let's try it again!

In [81]:
all_tags = {}

# Loop through every list
for tag_list in music_tags["tags"]:
    
    # Loop through every element in the list
    for tag in tag_list:
        # Add them to the dict or increment their frequency by 1
        if tag not in all_tags:
            all_tags[tag] = 1
        else:
            all_tags[tag] += 1

In [82]:
all_tags_sorted = {k: v for k, v in sorted(all_tags.items(), key=lambda item: item[1])}

Great, the data is now properly cleaned! <br>
In the next step, I'm going to get rid of this impractical column of lists. It would be more practical to create a boolean based dataframe with the tracks as rows and every unqiue tag as a column.

# 3. Transformation

Using a smaller test dataset, I have written the two functions below. get_all_unique() will return all unique tags as a list. Create_boolean_df() will us these as columns and every track as rows. It will use True and False values to state whether a tag (column) is in a track (row).

In [83]:
def get_all_unique(dataframe_of_lists):
    # Create empty list
    unique_tags = []
    
    # Loop through the dataframe rows (lists) and each item inside
    for row in dataframe_of_lists:
        for item in row:
            
            # Add item to all_tags if it's not already in it
            if item not in unique_tags:
                unique_tags.append(item)
                
    return unique_tags

In [84]:
def create_boolean_df(unique_tags, track_taglists):
    # Create new df with a column for every tag
    boolean_df = pd.DataFrame(columns = unique_tags)
    boolean_df.shape
    
    # Create an empty dict
    data_dict = {}
    
    # Loop through the columns (tags) in the boolean_df and add them to the dict
    for col in boolean_df:
        data_dict[col] = []
        
        # Loop through the taglists in the old dataframe
        for taglist in track_taglists:
            
            # Check if the column (tag) is in the tracks taglist. If yes append True else append False
            data_dict[col].append(col in taglist)
    
    # Use the boolean lists as values for the boolean_df's columns
    for col in boolean_df:
        boolean_df[col] = data_dict[col]
        
    return boolean_df

Let's use these functions to get a boolean based representation fo our data.

In [85]:
unique_tags = get_all_unique(music_tags["tags"])

In [86]:
music_tags_bool = create_boolean_df(unique_tags, music_tags["tags"])

In [87]:
music_tags_bool.shape

(253, 917)

In [88]:
music_tags_bool.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Columns: 917 entries, ukulele to holiday
dtypes: bool(917)
memory usage: 226.6 KB


Now, all we need to do is add the track_names as indices for the new df and then store it as a csv file.

In [89]:
music_tags_bool = music_tags_bool.set_index(music_tags["track_name"])

In [90]:
music_tags_bool.head()

,ukulele,happy,funny,advertising,upbeat,kid,kids,positive,chidren,joy,...,shangai,koto,guzheng,erhu,dizi,voice,sfx,discover,geek,holiday
track_name,,,,,,,,,,,,,,,,,,,,,
ukulele,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
creative minds,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
a new beginning,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
little idea,False,True,True,False,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
jazzy frenchy,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [91]:
music_tags_bool.to_csv("music_tags_bool.csv")